In [9]:
#Nome do arquivo excel com correspondência dos pontos (National)
XL_National="Lista_Pontos - National - Exemplo.xlsx"
XL_CSI="Lista_pontos - CSI - Exemplo.xlsx"

#Caminho do diretório onde estão os DADOS, a partir do diretório atual
Data_National='/Dados/National'
Data_CSI='/Dados/CSI'

#Caminho do diretório onde estão as FOTOS, a partir do diretório atual
Data_Fotos='/Fotos'


In [11]:
print('------Início------')
import numpy as np
import matplotlib.pyplot as plt
import xlrd, os, time, datetime
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import CellIsRule
from openpyxl import load_workbook
import pandas as pd
from scipy.signal import find_peaks as fpeaks
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_ALIGN_PARAGRAPH, WD_BREAK
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx import Document
from docx.shared import Inches
print('------Pacotes carregados------')

print('Modificação AATB')

start_time = time.time()
P_Proc=0 # Numero de pontos processados

# Cria variável com o caminho da pasta atual
my_path=os.getcwd()

XLName=XL_National

# Abre a planilha excel como workbook para leitura 'xlrd'
File=xlrd.open_workbook(XLName)

# Cria lista com os nomes das abas da planilha aberta
SheetNames=File.sheet_names()

S_Resumo=File.sheet_by_name('Resumo')


document = Document()

document.add_heading('Preliminary Vibration Report', 0)

p = document.add_paragraph('This report was generated from vibration measurement data acquired at RPBC '+
                           'on May 27th 2019.')

p = document.add_paragraph('This preliminary report presents the overall vibration values taken from piping and vessels as well '+
                          'as the spectra of those point exceeding 19mm/s RMS overall value.')


                          
#p.add_run('bold').bold = True
#p.add_run(' and some ')
#p.add_run('italic.').italic = True



document.add_heading('Measurement Case: ', level=1)



document.add_paragraph('Table 1 - Overall vibration values in mm/s RMS.')
last_paragraph = document.paragraphs[-1] 
last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

table = document.add_table(rows=1, cols=4)

hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Point'
hdr_cells[1].text = 'N-S'
hdr_cells[2].text = 'L-O'
hdr_cells[3].text = 'V'


table.alignment = WD_TABLE_ALIGNMENT.CENTER
table.autofit=True
table.style='TableGrid'

p=document.add_paragraph(' ')

run=p.add_run()
run.add_break(WD_BREAK.PAGE)


plt.figure(figsize=(6.1, 4.1), dpi=160, facecolor='w', edgecolor='k')

Global_dict = {'Ponto': ['N-S','L-O','Vert']}
Fig_dict = {'Ponto': ['N-S','L-O','Vert']}
Direc={'N-S':0,'L-O':1,'Vert':2}

Overload_detect=False

Descarte=0
Fig=0

k=2

Foto_test={'TAG':True}

while k < S_Resumo.nrows:

    if int(S_Resumo.cell(k,3).value) == 1:
        
        Report_file_name=S_Resumo.cell(k,2).value
        Chosen_s=S_Resumo.cell(k,1).value
        print('-- Processando dados de ' + Chosen_s + ' --')

        
        DirName1 = my_path + '/Espectros/National' + Report_file_name
        
        if not os.path.exists(DirName1):
            os.makedirs(DirName1)

        DirName2 = my_path + '/Waveform/National' + Report_file_name
        if not os.path.exists(DirName2):
            os.makedirs(DirName2)
        
        DirName3 = my_path + Data_Fotos+'/'

        # Cria variável 'Sheet' com a aba desejada
        Sheet=File.sheet_by_name(Chosen_s)
        
        aux=4


        Global=[]

        N_Pontos=Sheet.nrows-4

        for aux in range(4,N_Pontos+4,1):
            Global.append([Sheet.cell(aux, dir_aux).value for dir_aux in range(7, 11, 1)])



        # Lê da aba selecionada a data e o caso de operação informados
        Data_f=Sheet.cell(1,1).value
        Data_str=str(datetime.datetime(*xlrd.xldate_as_tuple(Data_f,File.datemode)))
        Data_str=Data_str[0:9]
        Caso=Sheet.cell(0,1).value

        ## Cria as curvas limites da ISO para vibração
        wlim=np.arange(0,500,1)

        Lim0=np.zeros(len(wlim))
        Lim1=np.zeros(len(wlim))
        Lim2=np.zeros(len(wlim))
        Lim3=np.zeros(len(wlim))

#         Lim1[1:len(wlim)]=10**((np.log10(wlim[1:len(wlim)])+0.48017)/2.127612)
#         Lim1[1:len(wlim)]=10**((np.log10(wlim[1:len(wlim)])+0.48017)/2.127612)
        Lim1[0]=0
        
        
#         Lim2[1:len(wlim)]=10**((np.log10(wlim[1:len(wlim)])+1.871083)/2.084547)
        Lim2[0]=0
        Lim0[0]=0
        Lim3[0]=0
    
    
#       limites do SWRI
        Lim0[1:len(wlim)] = 0.8808*wlim[1:len(wlim)]**0.5059 #Design
        Lim1[1:len(wlim)] = 1.8299*wlim[1:len(wlim)]**0.5062 #Marginal 
        Lim2[1:len(wlim)] = 3.4924*wlim[1:len(wlim)]**0.5067 #Correction
        Lim3[1:len(wlim)] = 8.9266*wlim[1:len(wlim)]**0.5062 #Danger

        i=4
        
        
        while (i-4 < N_Pontos):#(Sheet.cell(i,4).value != ""):
            
            if round(20*((i-4)/N_Pontos)) != round(20*((i-3)/N_Pontos)):
                print('--- ' + str(round(100*((i-4)/N_Pontos))) + '% Concluídos em ' + Chosen_s + ' ---')

            if (Sheet.cell(i,0).value == 1): # Teste para linhas ativas

                # Lê na linha atual a frequência de corte, se houver
                if Sheet.cell(i, 4).value != '':
                    f_corte = Sheet.cell(i, 4).value
                else:
                    f_corte = 0

                for Ch in range(3,5,1):
                    
                    if Sheet.cell(i, 2 + Ch).value!='' and Sheet.cell(i, 2 + Ch).value!='-':
                    
                        Ponto = Sheet.cell(i, 1).value + ' ' + Sheet.cell(i, 2 + Ch).value  # Código do ponto de medição

                        First=True
                        for k in range(int(Sheet.cell(i,2).value)+1,int(Sheet.cell(i,3).value)+1,1):

                            Filename=my_path+Data_National+'/input' + str(Ch) + '(t) ' + str(k) + '.txt' # Nome do arquivo com os dados de vibração

                            # Lê e aplica às variáveis v e t os valores de vibração e tempo no arquivo texto 'Filename'
                            v_aux = np.loadtxt(Filename, skiprows=32, usecols=(1))*9.81*1000

                            if Overload_detect:
    #                             print(np.mean(v_aux))
                                if np.mean(v_aux) > 0.5:
                                    calc=False
                                    Descarte=Descarte+1
                                else:
                                    calc=True
                            else:
                                calc=True


                            if First & calc:
                                # print(v_aux)
                                N_aux = len(v_aux)  # Número de linhas do arquivo lido

                                t_aux = np.loadtxt(Filename, skiprows=32, usecols=(0))

                                dt = t_aux[1]
                                tf = t_aux[N_aux - 1]
                                # print(tf)
                                # print(dt)
                                dw = 1 / tf
                                w_max = 1 / (dt * 2)
                                w = np.arange(0, w_max, dw)

                                i_corte = int(np.round(f_corte / dw))

                                Spec_or=(np.fft.fft(v_aux))
                                Spec_or[0:i_corte]=0

                                Spec=2/N_aux*np.absolute(Spec_or)
                                First=False


                            elif calc:

                                Spec_or=np.fft.fft(v_aux)
                                v_plot=v_aux
                                Spec_or[0:i_corte] = 0

                                Spec=(Spec+2/N_aux*np.absolute(Spec_or))/2


                        # Spec=2/N_aux*np.absolute(Spec_or)
                    ## Cálculo do espectro desprezando os valores abaixo da frequência de corte

                    ## Integra o sinal no domínio da frequência caso a célula D1 da planilha seja 1
                    Spec[1:len(w)]=np.divide(Spec[1:len(w)],(w[1:len(w)]*2*np.pi))

                    Global_aux = np.sqrt(np.sum((Spec[0:len(w)] / np.sqrt(2)) ** 2))
                    
                    plot_spec=True
                    try:
                        
                        aux=Global_dict[Sheet.cell(i, 1).value][Direc[Sheet.cell(i, 2+Ch).value]]
                        if aux=='-':
                            Global_dict[Sheet.cell(i, 1).value][Direc[Sheet.cell(i, 2+Ch).value]]=Global_aux
                        else:
                            Global_dict[Sheet.cell(i, 1).value][Direc[Sheet.cell(i, 2+Ch).value]]=max([aux,Global_aux])
                            plot_spec=False
                        
                        
                    except:
                        if Sheet.cell(i, 2+Ch).value!='' and Sheet.cell(i, 2+Ch).value!='-':
                            Global_dict[Sheet.cell(i, 1).value]=['-','-','-']
                            Global_dict[Sheet.cell(i, 1).value][Direc[Sheet.cell(i, 2+Ch).value]]=Global_aux
                            Fig_dict[Sheet.cell(i, 1).value]=['-','-','-']
                        

                    ## Cálculo do valor global pelo espectro
                    if Sheet.cell(i, 2+Ch).value!='' and Sheet.cell(i, 2+Ch).value!='-':
                        if Ch == 3:
                            Global[i-4][Direc[Sheet.cell(i,2+Ch).value]]=Global_aux

                        else:
                            if Sheet.cell(i,5).value != Sheet.cell(i,6).value:
                                Global[i - 4][Direc[Sheet.cell(i, 2 + Ch).value]] = Global_aux
                            else:
                                Global[i - 4][3] = Global_aux



                    if Global_aux > 19:
                        print('Valor Global: ', Global_aux, 'mm/s em ', Ponto)

                    # Mostra o valor do valor global se for maior que 19 mm/s

                    ## Plota o espectro de vibração
                    plt.plot(w,Spec[0:len(w)]/np.sqrt(2),'b-',linewidth=1)

                    ## Configura limites dos eixos
                    plt.xlim(left=0, right=300)
                    plt.ylim(bottom=0, top=50)

                    ## Configura legendas dos eixos
                    plt.xlabel('Frequency (Hz)')
                    plt.ylabel('Amplitude (mm/s RMS - int)') ## Indicação de variável integrada na legenda do eixo
                    
                    auxi,Peaks=fpeaks(Spec[0:len(w)],height=0.7,distance=np.round(5.5/dw))
                    w_peaks=w[auxi]
                    Peaks=Peaks['peak_heights']
                    i_sort=np.argsort(Peaks)[::-1]
                    Peaks=Peaks[i_sort]
                    w_peaks=w_peaks[i_sort]
                    plt.plot(w_peaks[0:5],Peaks[0:5]/np.sqrt(2),'rx',linewidth=0.5)
                    
                    
                    for jj in range(5):
                        try:
                            plt.text(200,45-jj*2,str(np.around(Peaks[jj],decimals=1))+'mm/s @ '+str(np.round(w_peaks[jj],decimals=1))+'Hz',size=9)
                        except:
                            plt.text(200,40,'No Peaks!',size=9)

                    ## Plota as linhas limite da ISO
                    plt.plot(wlim, Lim0, 'g-', linewidth=0.5)
                    plt.plot(wlim, Lim1, 'b-', linewidth=0.5)
                    plt.plot(wlim, Lim2, 'y-', linewidth=0.5)
                    plt.plot(wlim, Lim3, 'r-', linewidth=0.5)

                    ## Cria o título do gráfico
                    plt.title(Data_str + ' / ' + Ponto + '  /  ' + str(np.round(Global_aux * 10) / 10) + 'mm/s RMS  /  ' + Caso)

                    ## Cria os textos identificando as áreas do gráfico
                    plt.text(230,7,'Design',size=14,bbox=dict(boxstyle="round",
                                                                 ec=(0,1,0),
                                                                 fc=(0,1,0),
                                                                 ))
                    
                    plt.text(200,18,'Marginal',size=14,bbox=dict(boxstyle="round",
                                                                 ec=(0,0,1),
                                                                 fc=(0,0,1),
                                                                 ))

                    plt.text(125,30,'Correction',size=14,bbox=dict(boxstyle="round",
                                                                 ec=(1,1,0),
                                                                 fc=(1,1,0),
                                                                 ))
                    
                    plt.text(50,42,'Danger',size=14,bbox=dict(boxstyle="round",
                                                                 ec=(1,0,0),
                                                                 fc=(1,0,0),
                                                                 ))
                    # plt.text(1, 45, 'VÉY!', size=15, weight="bold", bbox=dict(boxstyle="round",
                    #                                                     ec=(1, 0, 0),
                    #                                                     fc=(1, 0, 0),
                    #                                                     ))

                    ## Cria o nome do arquivo do gráfico e salva no formato .jpg



                    FigName=DirName1 + '/' + Ponto + '.jpg'
                    plt.savefig(FigName)
                    
                    if Global_aux > 19 and plot_spec:
                               
                        try:
                            aux_t=Foto_test[Sheet.cell(i, 1).value]
                        except:
                            
                            Fig=Fig+1
                            try:
                                document.add_picture(DirName3+Sheet.cell(i, 15).value, width=Inches(5))
                                last_paragraph = document.paragraphs[-1] 
                                last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                                document.add_paragraph('Figure '+str(Fig)+' - Measurements on point: '+
                                                       Sheet.cell(i, 1).value+'.',style='Caption')
                                last_paragraph = document.paragraphs[-1] 
                                last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                                Foto_test[Sheet.cell(i, 1).value]=str(Fig)
                            except:
                                Fig=Fig-1
                        
                        Fig=Fig+1
                        Fig_dict[Sheet.cell(i, 1).value][Direc[Sheet.cell(i, 2+Ch).value]]=str(Fig)
                        document.add_picture(FigName, width=Inches(6))
                        last_paragraph = document.paragraphs[-1] 
                        last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                        document.add_paragraph('Figure '+str(Fig)+' - Vibration spectrum from point '+
                                               Sheet.cell(i, 1).value + ' in the direction ' + Sheet.cell(i, 2 + Ch).value+'.',style='Caption')
                        last_paragraph = document.paragraphs[-1] 
                        last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                        

                    ## Limpa o gráfico
                    plt.clf()

                    P_Proc=P_Proc+1

                    # Teste para plot de waveform
                    if (Sheet.cell(i,11).value == 1):

                        ## Integra o sinal no tempo, caso necessário
                        # if (Sheet.cell(0, 3).value == 1):

                        # v_plot=np.fft.ifft(np.real(Spec_or))
                        v1_i=0*v_plot
                        for j in range(1,N_aux-1):
                            v1_i[j]=np.sum(v_plot[0:j+1]*dt)

                            # v1=v1_i
                        v_plot=v1_i
                        ## Plota o waveform
                        plt.plot(t_aux,v_plot, 'k-', linewidth=0.5)

                        ## Configura as demais características do gráfico
                        plt.xlabel('Time (s)')
                        plt.ylabel('Amplitude (mm/s)')
                        plt.title(
                            Data_str + ' / ' + Ponto + '  /  ' + str(np.round(Global_aux * 10) / 10) + 'mm/s RMS  /  ' + Caso)
                        plt.xlim(left=0, right=tf)



                        FigName = DirName2 + '/' + Ponto + '_WF.jpg'
                        plt.savefig(FigName)

                        plt.clf()


                
                i=i+1
            else:
                i=i+1

        
        
        Point_list=list(Global_dict.keys())
        
        for t in Point_list[1:len(Point_list)]:
            
            row_cells = table.add_row().cells
                
            try:    
                row_cells[0].text = t+' (Fig.'+Foto_test[t]+')'
            except: 
                row_cells[0].text = t
            
            if type(Global_dict[t][0])==np.float64:
                row_cells[1].text = str(np.around(Global_dict[t][0],decimals=1))
                if Global_dict[t][0]>19:
                    row_cells[1].text=row_cells[1].text+' (Fig. '+Fig_dict[t][0]+')'
                    row_cells[1]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FFFF00"/>'.format(nsdecls('w'))))
                    if Global_dict[t][0]>27.5:
                        row_cells[1]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FF0000"/>'.format(nsdecls('w'))))
            else:
                row_cells[1].text = str(Global_dict[t][0])

            if type(Global_dict[t][1])==np.float64:
                row_cells[2].text = str(np.around(Global_dict[t][1],decimals=1))
                if Global_dict[t][1]>19:
                    row_cells[2].text=row_cells[2].text+' (Fig. '+Fig_dict[t][1]+')'
                    row_cells[2]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FFFF00"/>'.format(nsdecls('w'))))
                    if Global_dict[t][1]>27.5:
                        row_cells[2]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FF0000"/>'.format(nsdecls('w'))))
            else:
                row_cells[2].text = str(Global_dict[t][1])

            if type(Global_dict[t][2])==np.float64:
                row_cells[3].text = str(np.around(Global_dict[t][2],decimals=1))
                if Global_dict[t][2]>19:
                    row_cells[3].text=row_cells[3].text+' (Fig. '+Fig_dict[t][2]+')'
                    row_cells[3]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FFFF00"/>'.format(nsdecls('w'))))
                    if Global_dict[t][2]>27.5:
                        row_cells[3]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FF0000"/>'.format(nsdecls('w'))))
            else:
                row_cells[3].text = str(Global_dict[t][2])
        
        
        
        
        
        ## Rotina para escrever os valores globais na planilha Excel

        ## Abre a planilha como workbook usando load_workbook do openpyxl
        book=load_workbook(XLName)

        ## Abre a planilha com permissão para escrita com ExcelWriter
        writer=pd.ExcelWriter(XLName,engine='openpyxl') # Este comando abre a planilha como planilha vazia

        ## Escreve na planilha aberta os valores lidos da planilha original
        writer.book=book

        ## Cria dicionario com os nomes das abas
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

        ## Cria DataFrame com os valores globais calculados
        Global_NS = list([Global[0][0]])
        #type(Global_NS)
        Global_LO =list([Global[0][1]])
        Global_V = list([Global[0][2]])
        Global_Md = list([Global[0][3]])
        
        
        
        

        for jj in range(1,N_Pontos,1):
            Global_NS.append(Global[jj][0])
            Global_LO.append(Global[jj][1])
            Global_V.append(Global[jj][2])
            Global_Md.append(Global[jj][3])

        df=pd.DataFrame({'1':Global_NS,'2':Global_LO,'3':Global_V,'4':Global_Md})
        # print(df)

        ## Escreve os valores do DataFrame na planilha
        df.to_excel(writer,sheet_name=Chosen_s,startrow=4,startcol=7,header=False,index=False)

        # print(df)

        ## Cria variável com a aba selecionada
        ws=book[Chosen_s]

        ## Cria formatações condicionais nas células dos valores globais
        greenFill = PatternFill(start_color='00CC00',
                       end_color='00CC00',
                        fill_type='solid')

        ws.conditional_formatting.add('H5:K'+str(N_Pontos+4),
                     CellIsRule(operator='between', formula=[12.5,19], fill=greenFill))

        yelFill = PatternFill(start_color='FFFF00',
                       end_color='FFFF00',
                        fill_type='solid')

        ws.conditional_formatting.add('H5:K'+str(N_Pontos+4),
                     CellIsRule(operator='between', formula=[19,27.5], fill=yelFill))

        redFill = PatternFill(start_color='FF0000',
                       end_color='FF0000',
                        fill_type='solid')

        ws.conditional_formatting.add('H5:K'+str(N_Pontos+4),
                     CellIsRule(operator='between', formula=[27.5,1e10], fill=redFill))
        k=k+1

    else:
        k=k+1

writer.save()

# book.save(XLName)
# book.close()
## Escreve o tempo decorrido
plt.close('all')
print(' ')
Proc_time=round(10*(time.time()-start_time))/10
print('---  ' + str(P_Proc) + ' Pontos processados em ' + str(Proc_time) + ' segundos ---')
print(' ')
print('---  ' + str(Descarte) + ' Medições descartadas ---')
document.paragraphs[3].text='Caso de medição: '+Report_file_name
document.save(Report_file_name+'.docx')

------Início------
------Pacotes carregados------
-- Processando dados de Lista1 --
--- 0% Concluídos em Lista1 ---
--- 25% Concluídos em Lista1 ---
Valor Global:  24.457329992581073 mm/s em  Ponto 2 L-O
--- 50% Concluídos em Lista1 ---
Valor Global:  25.857199975955158 mm/s em  Ponto 3 L-O
--- 75% Concluídos em Lista1 ---
 
---  8 Pontos processados em 11.3 segundos ---
 
---  0 Medições descartadas ---


In [8]:
print('------Início------')
import numpy as np
import matplotlib.pyplot as plt
import xlrd, os, time, datetime
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import CellIsRule
from openpyxl import load_workbook
import pandas as pd
print('------Pacotes carregados------')

start_time = time.time()
P_Proc=0 # Numero de pontos processados

# Cria variável com o caminho da pasta atual
my_path=os.getcwd()

XLName=XL_CSI

# Abre a planilha excel como workbook para leitura 'xlrd'
File=xlrd.open_workbook(XLName)


# Cria lista com os nomes das abas da planilha aberta
SheetNames=File.sheet_names()

S_Resumo=File.sheet_by_name('Resumo')

Global_dict = {'Ponto': ['N-S','L-O','Vert']}
Fig_dict = {'Ponto': ['N-S','L-O','Vert']}
TAG = {'NS': 0,
       'LO': 1,
       'V': 2}

plt.figure(figsize=(6.1, 4.1), dpi=160, facecolor='w', edgecolor='k')

k=2

Lim0=np.zeros(len(wlim))
Lim1=np.zeros(len(wlim))
Lim2=np.zeros(len(wlim))
Lim3=np.zeros(len(wlim))

#         Lim1[1:len(wlim)]=10**((np.log10(wlim[1:len(wlim)])+0.48017)/2.127612)
#         Lim1[1:len(wlim)]=10**((np.log10(wlim[1:len(wlim)])+0.48017)/2.127612)
Lim1[0]=0


#         Lim2[1:len(wlim)]=10**((np.log10(wlim[1:len(wlim)])+1.871083)/2.084547)
Lim2[0]=0
Lim0[0]=0
Lim3[0]=0


#       limites do SWRI
Lim0[1:len(wlim)] = 0.8808*wlim[1:len(wlim)]**0.5059 #Design
Lim1[1:len(wlim)] = 1.8299*wlim[1:len(wlim)]**0.5062 #Marginal 
Lim2[1:len(wlim)] = 3.4924*wlim[1:len(wlim)]**0.5067 #Correction
Lim3[1:len(wlim)] = 8.9266*wlim[1:len(wlim)]**0.5062 #Danger
        
        
while S_Resumo.cell(k,1).value != '':

    if int(S_Resumo.cell(k,3).value) == 1:

        Chosen_s=S_Resumo.cell(k,1).value
        print('-- Processando dados de ' + Chosen_s + ' --')

        DirName1 = my_path + '/Espectros/CSI/' + Report_file_name
        if not os.path.exists(DirName1):
            os.makedirs(DirName1)

        DirName2 = my_path + '/Waveform/CSI/' + Report_file_name
        if not os.path.exists(DirName2):
            os.makedirs(DirName2)


        # Cria variável 'Sheet' com a aba desejada
        Sheet=File.sheet_by_name(Chosen_s)

        aux=4

        # Conta quantas linhas a aba selecionada tem e grava os valores globais já escritos, se existirem
        Global_aux=np.zeros(10000)
        while (Sheet.cell(aux,4).value != ""):
            if (Sheet.cell(aux,6).value != ''):
                Global_aux[aux-4]=Sheet.cell(aux,6).value
            aux=aux+1

        N_Pontos=aux-4

        Global=Global_aux[0:N_Pontos]

        # Lê da aba selecionada a data e o caso de operação informados
        Data_f=Sheet.cell(1,1).value
        Data_str=str(datetime.datetime(*xlrd.xldate_as_tuple(Data_f,File.datemode)))
        Data_str=Data_str[0:9]
        Caso=Sheet.cell(0,1).value

        ## Cria as curvas limites da ISO para vibração
        wlim=np.arange(0,500,1)

        

        i=4

        while (i-4 < N_Pontos):#(Sheet.cell(i,4).value != ""):
            if round(20*((i-4)/N_Pontos)) != round(20*((i-3)/N_Pontos)):
                print('--- ' + str(round(100*((i-4)/N_Pontos))) + '% Concluídos em ' + Chosen_s + ' ---')

            if (Sheet.cell(i,0).value == 1): # Teste para linhas ativas


                Filename=my_path+Data_CSI +'/'+ Sheet.cell(i,4).value + '.txt' # Nome do arquivo com os dados de vibração
                Ponto = Sheet.cell(i, 2).value + ' ' + Sheet.cell(i, 3).value # Código do ponto de medição

                # Lê na linha atual a frequência de corte, se houver
                if Sheet.cell(i,5).value != '':
                    f_corte=Sheet.cell(i,5).value
                else:
                    f_corte=0

                # Lê e aplica às variáveis v e t os valores de vibração e tempo no arquivo texto 'Filename'
                #print(Filename)
                v = np.loadtxt(Filename,skiprows=8,usecols=(1,3,5,7))
                t = np.loadtxt(Filename,skiprows=8,usecols=(0,2,4,6))/1000

                N1=len(v) # Número de linhas do arquivo lido
                cols=len(v[1,:]) # Número de colunas do arquivo lido
                N=N1*cols # Número total de elementos

                # Reorganização dos dados lidos em arrays unidimensionais
                v1=np.zeros(N)
                t1=np.zeros(N)

                v1[0:N1]=v[:,0]
                v1[N1:2*N1]=v[:,1]
                v1[2*N1:3*N1]=v[:,2]
                v1[3*N1:4*N1]=v[:,3]

                t1[0:N1]=t[:,0]
                t1[N1:2*N1]=t[:,1]
                t1[2*N1:3*N1]=t[:,2]
                t1[3*N1:4*N1]=t[:,3]

                ## Cálculo da vibração global pela série temporal
                # Global[i-4]=np.sqrt(1/N*np.sum(v1**2))

                ## Criação do array de frequências
                dt=t1[1]
                tf=t1[N-1]

                dw=1/tf
                w_max=1/(dt*2)
                w=np.arange(0,w_max,dw)

                ## Cálculo do espectro desprezando os valores abaixo da frequência de corte

                i_corte=int(np.round(f_corte/dw))

                Spec=2/N*(np.fft.fft(v1))
                Spec=np.absolute(Spec)
                Spec[0:i_corte]=0

                ## Integra o sinal no domínio da frequência caso a célula D1 da planilha seja 1
                if (Sheet.cell(0,3).value == 1):
                    Spec[1:len(w)]=9.81*np.divide(Spec[1:len(w)],(w[1:len(w)]*2*np.pi))*1000

                ## Cálculo do valor global pelo espectro
                Global[i-4]=np.sqrt(np.sum((Spec[0:len(w)]/np.sqrt(2))**2))
                
                
                # Mostra o valor do valor global se for maior que 19 mm/s
                if Global[i-4] > 19:
                    print('Valor Global: ',Global[i-4],'mm/s em ',Ponto)

                try:
                    aux=Global_dict[Sheet.cell(i, 2).value]
                    Global_dict[Sheet.cell(i, 2).value][TAG[Sheet.cell(i, 3).value]]=Global[i-4]
                except:
                    Global_dict[Sheet.cell(i, 2).value]=['-','-','-']
                    Global_dict[Sheet.cell(i, 2).value][TAG[Sheet.cell(i, 3).value]]=Global[i-4]
                    Fig_dict[Sheet.cell(i, 2).value]=['-','-','-']
               
               
                ## Plota o espectro de vibração
                plt.plot(w,Spec[0:len(w)]/np.sqrt(2),'b-',linewidth=1)
                
                plt.xlim(left=0, right=300)
                plt.ylim(bottom=0, top=50)

                ## Configura limites dos eixos
                plt.plot(wlim, Lim0, 'g-', linewidth=0.5)
                plt.plot(wlim, Lim1, 'b-', linewidth=0.5)
                plt.plot(wlim, Lim2, 'y-', linewidth=0.5)
                plt.plot(wlim, Lim3, 'r-', linewidth=0.5)

                ## Configura legendas dos eixos
                plt.xlabel('Frequency (Hz)')
                if (Sheet.cell(0, 3).value == 1):
                    plt.ylabel('Amplitude (mm/s RMS - int)') ## Indicação de variável integrada na legenda do eixo
                else:
                    plt.ylabel('Amplitude (mm/s RMS)')
                    
                auxi,Peaks=fpeaks(Spec[0:len(w)],height=0.7,distance=np.round(5.5/dw))
                w_peaks=w[auxi]
                Peaks=Peaks['peak_heights']
                i_sort=np.argsort(Peaks)[::-1]
                Peaks=Peaks[i_sort]
                w_peaks=w_peaks[i_sort]
                plt.plot(w_peaks[0:5],Peaks[0:5]/np.sqrt(2),'rx',linewidth=0.5)


                for jj in range(5):
                    try:
                        plt.text(200,45-jj*2,str(np.around(Peaks[jj],decimals=1))+'mm/s @ '+str(np.round(w_peaks[jj],decimals=1))+'Hz',size=9)
                    except:
                        plt.text(200,40,'No Peaks!',size=9)

                ## Plota as linhas limite da ISO
                plt.plot(wlim, Lim1, 'r-', linewidth=0.5)
                plt.plot(wlim, Lim2, 'r-', linewidth=0.5)

                ## Cria o título do gráfico
                plt.title(Data_str + ' / ' + Ponto + '  /  ' + str(np.round(Global[i - 4] * 10) / 10) + 'mm/s RMS  /  ' + Caso)

                ## Cria os textos identificando as áreas do gráfico
                plt.text(230,7,'Design',size=14,bbox=dict(boxstyle="round",
                                                                 ec=(0,1,0),
                                                                 fc=(0,1,0),
                                                                 ))
                    
                plt.text(200,18,'Marginal',size=14,bbox=dict(boxstyle="round",
                                                             ec=(0,0,1),
                                                             fc=(0,0,1),
                                                             ))

                plt.text(125,35,'Correction',size=14,bbox=dict(boxstyle="round",
                                                             ec=(1,1,0),
                                                             fc=(1,1,0),
                                                             ))

                plt.text(50,42,'Danger',size=14,bbox=dict(boxstyle="round",
                                                             ec=(1,0,0),
                                                             fc=(1,0,0),
                                                             ))
                # plt.text(1, 45, 'VÉY!', size=15, weight="bold", bbox=dict(boxstyle="round",
                #                                                     ec=(1, 0, 0),
                #                                                     fc=(1, 0, 0),
                #                                                     ))

                ## Cria o nome do arquivo do gráfico e salva no formato .jpg



                FigName=DirName1 + '/' + Ponto + '.jpg'
                plt.savefig(FigName)
                
                if Global[i-4] > 19:
                        Fig=Fig+1
                        document.add_picture(FigName, width=Inches(6))
                        last_paragraph = document.paragraphs[-1] 
                        last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER                        
                        
                        document.add_paragraph('Figure '+str(Fig)+' - Vibration spectrum from point '+
                                               Sheet.cell(i, 2).value + ' in the direction ' + Sheet.cell(i, 3).value+'.',style='Caption')
                        last_paragraph = document.paragraphs[-1] 
                        last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                        Fig_dict[Sheet.cell(i, 2).value][TAG[Sheet.cell(i, 3).value]]=str(Fig)

                ## Limpa o gráfico
                plt.clf()

                P_Proc=P_Proc+1

                ## Teste para plot de waveform
                if (Sheet.cell(i,7).value == 1):

                    ## Integra o sinal no tempo, caso necessário
                    if (Sheet.cell(0, 3).value == 1):
                        v1_i=0*v1
                        for j in range(1,N-1):
                            v1_i[j]=np.sum(v1[0:j+1]*dt)*9.81*1000

                        v1=v1_i

                    ## Plota o waveform
                    plt.plot(t1,v1, 'k-', linewidth=0.5)

                    ## Configura as demais características do gráfico
                    plt.xlabel('Time (s)')
                    plt.ylabel('Amplitude (mm/s)')
                    plt.title(
                        Data_str + ' / ' + Ponto + '  /  ' + str(np.round(Global[i - 4] * 10) / 10) + 'mm/s RMS  /  ' + Caso)
                    plt.xlim(left=0, right=tf)



                    FigName = DirName2 + '/' + Ponto + '_WF.jpg'
                    plt.savefig(FigName)

                    plt.clf()


                    
                    
                    
                    
                    
                i=i+1
            else:
                i=i+1
        
        Point_list=list(Global_dict.keys())
        
        for t in Point_list[1:len(Point_list)]:
            
            row_cells = table.add_row().cells
                
                
            row_cells[0].text = t
            
            
            
            if type(Global_dict[t][0])==np.float64:
                row_cells[1].text = str(np.around(Global_dict[t][0],decimals=1))
                if Global_dict[t][0]>19:
                    row_cells[1].text=row_cells[1].text+' (Fig. '+Fig_dict[t][0]+')'
                    row_cells[1]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FFFF00"/>'.format(nsdecls('w'))))
                    if Global_dict[t][0]>27.5:
                        row_cells[1]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FF0000"/>'.format(nsdecls('w'))))
            else:
                row_cells[1].text = str(Global_dict[t][0])

            if type(Global_dict[t][1])==np.float64:
                row_cells[2].text = str(np.around(Global_dict[t][1],decimals=1))
                if Global_dict[t][1]>19:
                    row_cells[2].text=row_cells[2].text+' (Fig. '+Fig_dict[t][1]+')'
                    row_cells[2]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FFFF00"/>'.format(nsdecls('w'))))
                    if Global_dict[t][1]>27.5:
                        row_cells[2]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FF0000"/>'.format(nsdecls('w'))))
            else:
                row_cells[2].text = str(Global_dict[t][1])

            if type(Global_dict[t][2])==np.float64:
                row_cells[3].text = str(np.around(Global_dict[t][2],decimals=1))
                if Global_dict[t][2]>19:
                    row_cells[3].text=row_cells[3].text+' (Fig. '+Fig_dict[t][2]+')'
                    row_cells[3]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FFFF00"/>'.format(nsdecls('w'))))
                    if Global_dict[t][2]>27.5:
                        row_cells[3]._tc.get_or_add_tcPr().append(parse_xml(r'<w:shd {} w:fill="FF0000"/>'.format(nsdecls('w'))))
            else:
                row_cells[3].text = str(Global_dict[t][2])

            

        ## Rotina para escrever os valores globais na planilha Excel

        ## Abre a planilha como workbook usando load_workbook do openpyxl
        book=load_workbook(XLName)

        ## Abre a planilha com permissão para escrita com ExcelWriter
        writer=pd.ExcelWriter(XLName,engine='openpyxl') # Este comando abre a planilha como planilha vazia

        ## Escreve na planilha aberta os valores lidos da planilha original
        writer.book=book

        ## Cria dicionario com os nomes das abas
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

        ## Cria DataFrame com os valores globais calculados
        df=pd.DataFrame({'Data':Global})

        ## Escreve os valores do DataFrame na planilha
        df.to_excel(writer,sheet_name=Chosen_s,startrow=4,startcol=6,header=False,index=False)

        ## Cria variável com a aba selecionada
        ws=book[Chosen_s]

        ## Cria formatações condicionais nas células dos valores globais
        greenFill = PatternFill(start_color='00CC00',
                       end_color='00CC00',
                        fill_type='solid')

        ws.conditional_formatting.add('G5:G'+str(i),
                     CellIsRule(operator='between', formula=[12.5,19], fill=greenFill))

        yelFill = PatternFill(start_color='FFFF00',
                       end_color='FFFF00',
                        fill_type='solid')

        ws.conditional_formatting.add('G5:G'+str(i),
                     CellIsRule(operator='between', formula=[19,27.5], fill=yelFill))

        redFill = PatternFill(start_color='FF0000',
                       end_color='FF0000',
                        fill_type='solid')

        ws.conditional_formatting.add('G5:G'+str(i),
                     CellIsRule(operator='greaterThan', formula=[27.5], fill=redFill))
        k=k+1

    else:
        k=k+1
plt.close('all')
writer.save()
# book.save(XLName)
# book.close()
## Escreve o tempo decorrido
print(' ')
Proc_time=round(10*(time.time()-start_time))/10
document.save(Report_file_name+'.docx')
print('---  ' + str(P_Proc) + ' Pontos processados em ' + str(Proc_time) + ' segundos ---')

------Início------
------Pacotes carregados------
-- Processando dados de Lista1 --
--- 0% Concluídos em Lista1 ---
--- 20% Concluídos em Lista1 ---
--- 40% Concluídos em Lista1 ---
--- 60% Concluídos em Lista1 ---
--- 80% Concluídos em Lista1 ---
 
---  5 Pontos processados em 9.8 segundos ---
